# Deploy Watson ML (PMML)

Deploys a PMML model to IBM Watson Machine Learning (WML)

In [1]:
!pip install ibm-watson-machine-learning==1.0.45
# PLEASE RESTART YOUR KERNAL AFTER THIS LINE HAS BEEN EXECUTED

     |████████████████████████████████| 1.7 MB 10.7 MB/s            
     |████████████████████████████████| 10.1 MB 31.5 MB/s            
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
     |████████████████████████████████| 51 kB 800 kB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 824 kB 55.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 133 kB 52.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 547 kB 32.8 MB/s            
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72565 sha256=ac5e77ef83e52258b4362807085c6eae603ceba1b43bb43025eccc924e8f591d
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/47/22/bf/e1154ff0f5de93cc477acd0ca69abfbb8b799c5b28a66b44c2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501014 sha256=28a1d6aa2a3b1db1

In [1]:
from ibm_watson_machine_learning import APIClient
import logging
import os
import re
import sys

In [6]:
# IBM Cloud API Key https://cloud.ibm.com/iam/apikeys
api_key = os.environ.get('api_key', 'aFIcZVwZpKZTcOxou-nrtbxCxHoSXURi-HrMLlWNAdSV')

# Machine Learning Model Deployment Space https://dataplatform.cloud.ibm.com/ml-runtime/spaces
space = os.environ.get('space', '83f2c612-5f6a-4021-a059-9f17488955ea')

# IBM Cloud Region (e.g. us-south)
location = os.environ.get('location', 'us-south')

# temporary directory for data
data_dir = os.environ.get('data_dir',
                          '../../data/')

In [7]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [8]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [9]:
client = APIClient(wml_credentials)

In [10]:
o = client.software_specifications.get_uid_by_name('spark-mllib_2.4')
software_spec_uid = o
client.set.default_space(space)

'SUCCESS'

In [11]:
model_meta_props = {
    client.repository.ModelMetaNames.NAME: 'test_pmml2',
    client.repository.ModelMetaNames.TYPE: "pmml_4.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
}

published_model = client.repository.store_model(
    model=data_dir + 'model.xml',
    meta_props=model_meta_props,
)

model_uid = client.repository.get_model_uid(published_model)